In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Activation, Dropout, Dense, Conv1D, GlobalMaxPool1D, MaxPool1D, Input, MaxPooling1D, Flatten
from tensorflow.keras.models import Model, Sequential
from keras.backend import clear_session
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.activations import tanh

from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from tensorflow import keras

import sys
sys.path.append("../")

from pre_processing import creazione_modello_GloVe

In [2]:
dataset = pickle.load(open("dataset/dataset_SLS.pickle", "rb"))

In [3]:
Y = np.array(dataset["sentiment"])

In [4]:
x_train,x_test,y_train,y_test = train_test_split(dataset["sentence"],Y, test_size=0.1, shuffle=True)

In [5]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(2700,) (2700,)
(300,) (300,)


In [6]:
print("numero di positivi nel train: ", list(y_train).count(1))
print("numero di negativi nel train: ", list(y_train).count(0))

print("numero di positivi nel test: ", list(y_test).count(1))
print("numero di negativi nel test: ", list(y_test).count(0))

numero di positivi nel train:  1360
numero di negativi nel train:  1340
numero di positivi nel test:  140
numero di negativi nel test:  160


In [7]:
embedding = creazione_modello_GloVe("dataset/glove.6B.50d.txt")

In [8]:
tokenizer = Tokenizer(num_words=140000)
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index

In [9]:
vocab_len = len(word_index)+1
embedding_vector_len = embedding["banana"].shape[0]
embedding_matrix = np.zeros((vocab_len, embedding_vector_len))

for word, index in word_index.items():
    vector = embedding.get(word)
    if vector is not None:
        embedding_matrix[index, :] = vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embedding_vector_len, input_length=300, weights=[embedding_matrix])

In [53]:
CNN_model = Sequential()
#CNN_model.add(Input(shape=(200,)))
CNN_model.add(embedding_layer)
CNN_model.add(Conv1D(filters=3, kernel_size=6, padding="same"))
CNN_model.add(MaxPooling1D(5, padding="same"))
CNN_model.add(Flatten())
#CNN_model.add(Dropout(0.2))
#CNN_model.add(Dense(32))
CNN_model.add(Dense(2, activation=tanh))
#RMSprop(learning_rate=0.20)
CNN_model.compile(loss="binary_crossentropy", optimizer=RMSprop(), metrics=["accuracy"])
CNN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 50)           247550    
_________________________________________________________________
conv1d (Conv1D)              (None, 300, 3)            903       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 60, 3)             0         
_________________________________________________________________
flatten (Flatten)            (None, 180)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 362       
Total params: 248,815
Trainable params: 248,815
Non-trainable params: 0
_________________________________________________________________


In [11]:
x_train_index = tokenizer.texts_to_sequences(x_train)
x_train_index = pad_sequences(x_train_index, maxlen=300)

In [12]:
y_train_categorical = keras.utils.to_categorical(y_train, 2)

In [54]:
clear_session()
CNN_model.fit(x_train_index[:2100], y_train_categorical[:2100], epochs=15, batch_size=32, verbose=1, validation_data=(x_train_index[2100:], y_train_categorical[2100:]))

Epoch 1/15
66/66 [==============================] - 1s 14ms/step - loss: 1.0986 - accuracy: 0.4995 - val_loss: 0.7930 - val_accuracy: 0.5117
Epoch 2/15
66/66 [==============================] - 1s 13ms/step - loss: 0.7615 - accuracy: 0.5281 - val_loss: 0.8023 - val_accuracy: 0.5217
Epoch 3/15
66/66 [==============================] - 1s 13ms/step - loss: 0.6927 - accuracy: 0.6162 - val_loss: 0.6959 - val_accuracy: 0.6050
Epoch 4/15
66/66 [==============================] - 1s 13ms/step - loss: 0.5669 - accuracy: 0.7524 - val_loss: 0.6821 - val_accuracy: 0.7133
Epoch 5/15
66/66 [==============================] - 1s 13ms/step - loss: 0.4127 - accuracy: 0.8705 - val_loss: 0.6589 - val_accuracy: 0.7883
Epoch 6/15
66/66 [==============================] - 1s 13ms/step - loss: 0.3074 - accuracy: 0.9148 - val_loss: 0.6932 - val_accuracy: 0.7683
Epoch 7/15
66/66 [==============================] - 1s 13ms/step - loss: 0.2452 - accuracy: 0.9390 - val_loss: 0.6870 - val_accuracy: 0.8067
Epoch 8/15
66

In [14]:
x_test_index = tokenizer.texts_to_sequences(x_test)
x_test_index = pad_sequences(x_test_index, maxlen=300)

In [55]:
y_pred = CNN_model.predict(x_test_index)

In [56]:
print(classification_report(y_test, np.argmax(y_pred, axis=1).astype("float32")))

              precision    recall  f1-score   support

           0       0.87      0.85      0.86       160
           1       0.83      0.86      0.85       140

    accuracy                           0.85       300
   macro avg       0.85      0.85      0.85       300
weighted avg       0.85      0.85      0.85       300



In [ ]:
CNN_model.save("SLS_CNN")